In [82]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load


# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/titanic/train.csv
/kaggle/input/titanic/test.csv
/kaggle/input/titanic/gender_submission.csv


Imports

In [83]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pylab as plt

from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

In [111]:
df_train = pd.read_csv('/kaggle/input/titanic/train.csv')
df_test = pd.read_csv('/kaggle/input/titanic/test.csv')
df_sub = pd.read_csv('../input/titanic/gender_submission.csv')

display(df_train)
display(df_test)
# display(df_sub)


,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


Okay heres the order for the MVP training set:
1. Drop non-MVP columns
2. Drop NaN values
3. Split X and Y
4. Fit
5. Score

1. Drop non-MVP columns from both train and test sets

In [113]:
train_copy = df_train.copy()
train_mvp = train_copy.drop(['PassengerId','Name','Sex','Ticket','Cabin',"Embarked"], axis=1)
display(train_mvp)

test_copy = df_test.copy()
test_mvp = test_copy.drop(['PassengerId','Name','Sex','Ticket','Cabin',"Embarked"], axis=1)
display(test_mvp)

,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...
886,0,2,27.0,0,0,13.0000
887,1,1,19.0,0,0,30.0000
888,0,3,NaN,1,2,23.4500
889,1,1,26.0,0,0,30.0000


,Pclass,Age,SibSp,Parch,Fare
0,3,34.5,0,0,7.8292
1,3,47.0,1,0,7.0000
2,2,62.0,0,0,9.6875
3,3,27.0,0,0,8.6625
4,3,22.0,1,1,12.2875
...,...,...,...,...,...
413,3,NaN,0,0,8.0500
414,1,39.0,0,0,108.9000
415,3,38.5,0,0,7.2500
416,3,NaN,0,0,8.0500


2. Drop NaN values


In [114]:
train_mvp = train_mvp.dropna(axis = 0)
train_mvp = train_mvp.reset_index(drop=True)
display(train_mvp)

test_mvp = test_mvp.dropna(axis = 0)
test_mvp = test_mvp.reset_index(drop=True)
display(test_mvp)


,Survived,Pclass,Age,SibSp,Parch,Fare
0,0,3,22.0,1,0,7.2500
1,1,1,38.0,1,0,71.2833
2,1,3,26.0,0,0,7.9250
3,1,1,35.0,1,0,53.1000
4,0,3,35.0,0,0,8.0500
...,...,...,...,...,...,...
709,0,3,39.0,0,5,29.1250
710,0,2,27.0,0,0,13.0000
711,1,1,19.0,0,0,30.0000
712,1,1,26.0,0,0,30.0000


,Pclass,Age,SibSp,Parch,Fare
0,3,34.5,0,0,7.8292
1,3,47.0,1,0,7.0000
2,2,62.0,0,0,9.6875
3,3,27.0,0,0,8.6625
4,3,22.0,1,1,12.2875
...,...,...,...,...,...
326,3,3.0,1,1,13.7750
327,1,37.0,1,0,90.0000
328,3,28.0,0,0,7.7750
329,1,39.0,0,0,108.9000


3. Split X and Y

In [95]:
X, Y = train_mvp.iloc[:,1:], train_mvp.iloc[:, 0]

display(X)
display(Y)

,Pclass,Age,SibSp,Parch,Fare
0,3,22.0,1,0,7.2500
1,1,38.0,1,0,71.2833
2,3,26.0,0,0,7.9250
3,1,35.0,1,0,53.1000
4,3,35.0,0,0,8.0500
...,...,...,...,...,...
709,3,39.0,0,5,29.1250
710,2,27.0,0,0,13.0000
711,1,19.0,0,0,30.0000
712,1,26.0,0,0,30.0000


0      0
1      1
2      1
3      1
4      0
      ..
709    0
710    0
711    1
712    1
713    0
Name: Survived, Length: 714, dtype: int64

4. Create Pipeline

In [96]:
pipe = Pipeline([("scaling", StandardScaler()), ("model", KNeighborsClassifier(n_neighbors=5))])
# pipe.get_params()

5. Create GridSearchCV

In [109]:
model = GridSearchCV(estimator = pipe, 
                     param_grid = {'model__n_neighbors': [6,7,8,9,10,11,12,13,14, 18, 22]},
                     cv=5)

6. Model.fit

In [110]:
model.fit(X,Y)
pd.DataFrame(model.cv_results_)

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_model__n_neighbors,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.006708,0.000761,0.009770,0.000503,6,{'model__n_neighbors': 6},0.629371,0.734266,0.727273,0.748252,0.704225,0.708677,0.042133,9
1,0.006278,0.000532,0.010036,0.000909,7,{'model__n_neighbors': 7},0.615385,0.699301,0.727273,0.762238,0.718310,0.704501,0.049013,11
2,0.006402,0.000641,0.009717,0.000487,8,{'model__n_neighbors': 8},0.643357,0.720280,0.734266,0.734266,0.711268,0.708687,0.033816,8
3,0.005854,0.000122,0.008873,0.000123,9,{'model__n_neighbors': 9},0.636364,0.706294,0.741259,0.769231,0.690141,0.708658,0.045402,10
4,0.005776,0.000127,0.008857,0.000077,10,{'model__n_neighbors': 10},0.629371,0.713287,0.748252,0.783217,0.711268,0.717079,0.051160,5
5,0.006464,0.000439,0.010075,0.000813,11,{'model__n_neighbors': 11},0.629371,0.713287,0.713287,0.797203,0.718310,0.714291,0.053111,7
6,0.006845,0.000489,0.010434,0.001139,12,{'model__n_neighbors': 12},0.636364,0.734266,0.734266,0.783217,0.718310,0.721284,0.047735,4
7,0.006322,0.000967,0.009194,0.000353,13,{'model__n_neighbors': 13},0.622378,0.720280,0.720280,0.776224,0.739437,0.715719,0.050946,6
8,0.005906,0.000224,0.009204,0.000313,14,{'model__n_neighbors': 14},0.636364,0.734266,0.734266,0.783217,0.739437,0.725510,0.048217,1
9,0.005862,0.000120,0.009333,0.000154,18,{'model__n_neighbors': 18},0.643357,0.727273,0.734266,0.783217,0.732394,0.724101,0.045159,2


In [116]:
df_preds = pd.DataFrame(model.predict(test_mvp))
display(df_preds.head(50))

,0
0,0
1,0
2,0
3,0
4,0
5,0
6,0
7,0
8,0
9,0
